In [1]:
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt

In [2]:
# Set a random seed for reproducibility
np.random.seed(4)

In [8]:
# Parameters
# Parameters
T = 1000  # Time-series length
n_x = 1  # Number of hidden states
n_w = n_x  # Number of process error terms

n_w2hat = n_x * (n_x + 1) / 2  # Total number of variance and covariance terms
y = np.zeros(T)  # Initialization of the vector of observations

# Q matrix
sW = np.sqrt(1.35)  # 0.42, 1.35, 18.75 - Possible values for the true error variances
Q = sW ** 2

# R matrix
QR_ratio = 10  # Q/R = (sigma_W)^2 / (sigma_V)^2
R = Q / QR_ratio
sV = np.sqrt(R)


In [10]:
# Data
YT = np.zeros((n_x, T))
x_true = np.zeros((n_x, T))
x_true[:, 0] = 0
w = np.dot(sW, np.random.randn(n_x, T))
v = np.dot(sV, np.random.randn(n_x, T))

for t in range(1, T):
    A_t = 0.8 - 0.1 * np.sin(7 * np.pi * t / T)  # Transition equation
    C_t = 1 - 0.99 * np.sin(100 * np.pi * t / T)  # Observation equation
    x_true[:, t] = A_t * x_true[:, t - 1] + w[:, t]
    YT[:, t] = C_t * x_true[:, t] + v[:, t]

In [ ]:
# Plot the data
plt.figure(figsize=(10, 5))
plt.plot(YT[0, :], label='Observations')
plt.show()

In [ ]:
# Initialization

mw2 = 2  # Initial mean for \overline{W^2}
sw2 = 1  # Initial variance for \overline{W^2}

EX = np.zeros((3, T))  # X = [X  W  \overline{W^2}]
EX[:, 0] = np.transpose([0, np.nan, mw2]) # Initial mean is zero
print(EX.shape)
PX = np.zeros((3, 3, T))
PX[:, :, 0] = np.diag([100, np.nan, sw2 ** 2])  # Initial variance is 100
print(PX[:, :, 0])

In [ ]:
for t in range(1, T):
    A = 0.8 - 0.1 * np.sin(7 * np.pi * t / T)  # Transition equation
    Ep = np.transpose([A * EX[0, t - 1], 0])  # Mean of W is zero
    Ep = Ep.reshape(Ep.shape[0], 1)
    m_w2hat = EX[2, t - 1]
    s_w_sq = m_w2hat

    Sp = np.array([[A ** 2 * PX[0, 0, t - 1] + s_w_sq, s_w_sq],
                   [s_w_sq, s_w_sq]])

    C = np.array([1 - 0.99 * np.sin(100 * np.pi * t / T), 0])  # Observation equation
    
    SY = C @ Sp @ C.T + R
    SYX = Sp @ C.T
    my = C @ Ep
    K = SYX / SY
    K = K.reshape(K.shape[0], 1)
    
    e = YT[:, t] - my
    e = e.reshape(e.shape[0], 1)
    SYX = SYX.reshape(SYX.shape[0], 1)
    
    EX1 = Ep + K @ e
    print(EX1)
    PX1 = Sp - K @ SYX.T
    # print(PX1)

    EX[0:2, t] = EX1
    PX[0:2, 0:2, t] = PX1
    v = YT[:, t] - my

    
